## Prediction of Daily PM10 and PM2.5

### Importing Data

The particulate matter data show some differences each other, on one hand the PM2.5 present fewer input variables as well some different type of variables if we compare these data with PM10.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA

from scipy.io import loadmat
from sklearn import preprocessing

from sklearn import linear_model

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.neural_network import MLPRegressor

from sklearn.metrics import max_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score



In [2]:
#Reading data and naming Columns
y_PM25=pd.read_csv('y_PM25', header=0, index_col=0, parse_dates=True)
X_PM25=pd.read_csv('X_PM25', header=0, index_col=0, parse_dates=True)
y_PM10=pd.read_csv('y_PM10', header=0, index_col=0, parse_dates=True)
X_PM10=pd.read_csv('X_PM10', header=0, index_col=0, parse_dates=True)

In [3]:
X_PM25.head()

,DEWP,TEMP,PRES,Iws,pm2.5_Max,DEWP_Max,TEMP_Max,PRES_Max,Iws_Max,pm2.5_Min,DEWP_Min,TEMP_Min,PRES_Min,Iws_Min
new_idx,,,,,,,,,,,,,,
2010-01-02,-8.500000,-5.125000,1024.750000,24.860000,181.0,-7,-4.0,1028.0,55.43,105.0,-16,-6.0,1020.0,1.79
2010-01-03,-10.125000,-8.541667,1022.791667,70.937917,107.0,-7,-6.0,1027.0,127.84,53.0,-12,-11.0,1020.0,0.89
2010-01-04,-20.875000,-11.500000,1029.291667,111.160833,79.0,-14,-9.0,1035.0,198.45,20.0,-26,-15.0,1023.0,16.09
2010-01-05,-24.583333,-14.458333,1033.625000,56.920000,106.0,-22,-11.0,1035.0,218.57,25.0,-27,-19.0,1031.0,0.89
2010-01-06,-23.708333,-12.541667,1033.750000,18.511667,132.0,-21,-8.0,1035.0,59.00,20.0,-26,-17.0,1033.0,0.89


In [4]:
#Standardization of data
scaler_X_PM10 = preprocessing.StandardScaler().fit(X_PM10.to_numpy())
scaler_y_PM10 = preprocessing.StandardScaler().fit(y_PM10.to_numpy())
scaler_X_PM25 = preprocessing.StandardScaler().fit(X_PM25.to_numpy())
scaler_y_PM25 = preprocessing.StandardScaler().fit(y_PM25.to_numpy())

#Other type of normalization=>
#MinMaxScaler #minmax_scale #MaxAbsScaler #StandardScaler #RobustScaler 
#Normalizer #QuantileTransformer #PowerTransformer
X_PM10_scaled=scaler_X_PM10.transform(X_PM10)
y_PM10_scaled=scaler_y_PM10.transform(y_PM10)

X_PM25_scaled=scaler_X_PM25.transform(X_PM25)
y_PM25_scaled=scaler_y_PM10.transform(y_PM25)

### Splitting data into Training and Testing samples - Cross-Validation

The data is split leaving some part out of the process, this part will be used to evualuate the performance of predictions. It is usually selected arround 70% or 80% of samples for learning the model.

In [5]:
#Percentage of Testing Data, Remainder is training data
splitter_PM10=0.10 
splitter_PM25=0.10 

#PCA Variance retained
sigma2_PM10=0.999
sigma2_PM25=0.999

In [6]:
#Splitting training and testing
X_train_PM10, X_test_PM10, y_train_PM10, y_test_PM10 = train_test_split(X_PM10_scaled, y_PM10_scaled, test_size=splitter_PM10, shuffle='True')
X_train_PM25, X_test_PM25, y_train_PM25, y_test_PM25 = train_test_split(X_PM25_scaled, y_PM25_scaled, test_size=splitter_PM25, shuffle='True')

In [7]:
#PCA PM10
#pca_PM10 = PCA(sigma2_PM10)
#pca_PM10 = pca_PM10.fit_transform(X_train_PM10)
#principalComponents_PM10 = pd.DataFrame(data = pca_PM10)
#print(len(principalComponents_PM10))

#PCA PM25
#pca_PM25 = PCA(sigma2_PM25)
#pca_PM25 = pca_PM25.fit_transform(X_train_PM25)
#principalComponents_PM25 = pd.DataFrame(data = pca_PM25)

In [8]:
#PM10 MLP
mlp_nn = MLPRegressor(hidden_layer_sizes=(16,32,32), activation='tanh', solver='lbfgs')
mlp_nn.fit(X_train_PM10, y_train_PM10.reshape(len(y_train_PM10),))

y_predict_train_PM10=mlp_nn.predict(X_train_PM10)
y_predict_test_PM10=mlp_nn.predict(X_test_PM10)

#Descalating outputs
y_predict_train_PM10_descaled=scaler_y_PM10.inverse_transform(y_predict_train_PM10)
y_train_PM10_descaled=scaler_y_PM10.inverse_transform(y_train_PM10)

y_predict_test_PM10_descaled=scaler_y_PM10.inverse_transform(y_predict_test_PM10)
y_test_PM10_descaled=scaler_y_PM10.inverse_transform(y_test_PM10)
    

In [9]:
#PM25 MLP
mlp_nn_PM25 = MLPRegressor(hidden_layer_sizes=(16,32,32), activation='tanh', solver='lbfgs')
mlp_nn_PM25.fit(X_train_PM25, y_train_PM25.reshape(len(y_train_PM25),))

y_predict_train_PM25=mlp_nn_PM25.predict(X_train_PM25)
y_predict_test_PM25=mlp_nn_PM25.predict(X_test_PM25)

#Descalating outputs
y_predict_train_PM25_descaled=scaler_y_PM25.inverse_transform(y_predict_train_PM25)
y_train_PM25_descaled=scaler_y_PM25.inverse_transform(y_train_PM25)
y_predict_test_PM25_descaled=scaler_y_PM25.inverse_transform(y_predict_test_PM25)
y_test_PM25_descaled=scaler_y_PM25.inverse_transform(y_test_PM25)


In [10]:
#Functions =>
def mean_absolute_percentage_error(y_true, y_pred): 
    #y_true, y_pred = check_arrays(y_true, y_pred)
    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def Wrap_Performance():
    lst=np.array(["Model","Layers","Optimization","MAE_Train(ug/m3)","MAE_Test(ug/m3)", "MAPE_Train(%)","MAPE_Test(%)"])
    Performance_PM10=pd.DataFrame(columns=lst)
    Performance_PM10["Model"]=["MLP_Regressor"]
    Performance_PM10["Layers"]=str(mlp_nn.hidden_layer_sizes)
    Performance_PM10["Optimization"]=["None"]  
    Performance_PM10["MAE_Train(ug/m3)"]=median_absolute_error(y_predict_train_PM10_descaled,y_train_PM10_descaled)
    Performance_PM10["MAE_Test(ug/m3)"]=median_absolute_error(y_predict_test_PM10_descaled,y_test_PM10_descaled)
    Performance_PM10["MAPE_Train(%)"]=mean_absolute_percentage_error(y_predict_train_PM10_descaled,y_train_PM10_descaled)
    Performance_PM10["MAPE_Test(%)"]=mean_absolute_percentage_error(y_predict_test_PM10_descaled,y_test_PM10_descaled)
    return Performance_PM10

def Wrap_Performance_PM25():
    lst=np.array(["Model","Layers","Optimization","MAE_Train(ug/m3)","MAE_Test(ug/m3)", "MAPE_Train(%)","MAPE_Test(%)"])
    Performance_PM25=pd.DataFrame(columns=lst)
    Performance_PM25["Model"]=["MLP_Regressor"]
    Performance_PM25["Layers"]=str(mlp_nn_PM25.hidden_layer_sizes)
    Performance_PM25["Optimization"]=["None"]
    Performance_PM25["MAE_Train(ug/m3)"]=median_absolute_error(y_predict_train_PM25_descaled,y_train_PM25_descaled)
    Performance_PM25["MAE_Test(ug/m3)"]=median_absolute_error(y_predict_test_PM25_descaled,y_test_PM25_descaled)
    Performance_PM25["MAPE_Train(%)"]=mean_absolute_percentage_error(y_predict_train_PM25_descaled,y_train_PM25_descaled)
    Performance_PM25["MAPE_Test(%)"]=mean_absolute_percentage_error(y_predict_test_PM25_descaled,y_test_PM25_descaled)
    return Performance_PM25

In [11]:
Performance_PM10=Wrap_Performance()
Performance_PM10

,Model,Layers,Optimization,MAE_Train(ug/m3),MAE_Test(ug/m3),MAPE_Train(%),MAPE_Test(%)
0,MLP_Regressor,"(16, 32, 32)",None,2.951163,9.806039,65.289201,80.041714


In [12]:
Performance_PM25=Wrap_Performance_PM25()
Performance_PM25

,Model,Layers,Optimization,MAE_Train(ug/m3),MAE_Test(ug/m3),MAPE_Train(%),MAPE_Test(%)
0,MLP_Regressor,"(16, 32, 32)",None,83.391581,124.768042,201.539498,166.718177
